<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/iTownpage_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# iタウンページからの緯度経度情報の抽出

### (1) 必要なライブラリの取得とインポート

In [ ]:
import os
os.environ["https_proxy"] = "http://cmproxy.nda.ac.jp:8080"
!pip install selenium --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd
from datetime import datetime, timedelta
import urllib.parse

import json
import time
import random

### (2) iタウンページへのアクセス
下記ボックス内の search_words, search_area に検索したい言葉と場所を設定してください。search_words にはスペース区切りで複数の単語を並べることが可能です。

In [ ]:
iTownPage = "https://itp.ne.jp"
headers = headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

search_words = 'ラーメン'
search_area = '横須賀市'

url = iTownPage + '/keyword/?keyword=' + search_words + '&areaword=' + search_area + '&sort=01&sbmap=false'
url

In [ ]:
# ブラウザを実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
#options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')

In [ ]:
#インスタンス化
driver = webdriver.Chrome(options=options)
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
# ページの取得
driver.get(url)

In [ ]:
res = driver.find_element(By.ID, "comp-lgurnb931")
hits = int(res.text[:-1])
print("「", search_area, "」に", hits, "件の「", search_words, "」が見つかりました", sep="")

### (3) 必要な情報の抽出

In [ ]:
cols = ['name', 'tel', 'address', 'lon', 'lat', 'lon_Tokyo', 'lat_Tokyo']
df = pd.DataFrame(columns=cols)

for i in range(0, hits, 20):
    url2 = 'https://itp.ne.jp/keyword/?area=&areaword=' + search_area + '&keyword=' + search_words + '&sbmap=false&sort=01&from=' + str(i)
    print(url2)
    driver.get(url2)
    for j in range(200):
        driver.execute_script('window.scrollBy(0, 20);')
        time.sleep(0.1)
    response = driver.find_element(By.ID, "wix-warmup-data")
    json_text = response.get_attribute('innerHTML')
    json_data = json.loads(json_text)
    di = json_data['appsWarmupData']['675bbcef-18d8-41f5-800e-131ec9e08762']
    dj = di[list(di.keys())[1]]['body']['hits']['hits']
    for j in dj:
        k = j['_source']['ki']
        l = [k['name'], k['tel1'], k['jusyo']+k['jyusyo_banti'], k['location_w'][0], k['location_w'][1], k['location'][0], k['location'][1]]
        #print(l)
        df2 = pd.DataFrame(l).T
        df2.columns = cols
        df = pd.concat([df, df2])

### (4) 抽出結果の確認

In [ ]:
df

### (5) 結果をファイルへ出力
このファイルと同じ場所に「検索語_検索場所.csv」というファイルができる。

In [ ]:
df.to_csv('_'.join(search_words.split()) + '_' + search_area + '.csv', index=False, encoding="cp932")